# 高専シラバスから科目一覧を作成

In [1]:
import sys
sys.path.append('/opt/python/lib/python3.11/site-packages')

In [2]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

## ページの取得
- 取得元：「高専Webシラバス」の「木更津高専」の「情報工学科」の「本年度の開校科目一覧」

- `header`と`page`と`param`を設定

In [3]:
header = {"User-Agent":"Mozilla/5.0"}
page = "https://syllabus.kosen-k.go.jp/Pages/PublicSubjects"
param = {"school_id":11,"department_id":"12","year":2025,"lang":"ja"}
r = get(page, params=param, headers=header)
r

<Response [200]>

## HTMLのパーズ
- `BeautifulSoup`を利用

In [5]:
soup = BeautifulSoup(r.text,"html.parser")

## データを抽出
- ブラウザでHTMLのソースコードを表示し、抽出すべき箇所を調べる
- 抽出した結果を見ながら、徐々に、抽出する条件を洗練してゆく
- 抽出結果からテキスト部分を抽出する

### `table`タグを抽出
- いくつ抽出された？

In [6]:
tag = 'table'
tables = soup.find_all(tag)
len(tables)

3

In [7]:
tables[2]

<table _fixedhead="rows:3; cols:3" border="1" cellpadding="0" cellspacing="0" id="sytablenc">
<tbody>
<tr class="ctop">
<td colspan="2" rowspan="4">科目区分</td>
<td rowspan="4">授業科目</td>
<td rowspan="4">科目番号</td>
<td rowspan="4">単位種別</td>
<td rowspan="4">単位数</td>
<td align="center" colspan="20" id="MainContent_tableTh">学年別週当授業時数</td>
<td rowspan="4">担当教員</td>
<td rowspan="4">履修上の区分</td>
</tr>
<tr>
<td class="c1t" colspan="4">1年</td>
<td class="c2t" colspan="4">2年</td>
<td class="c3t" colspan="4">3年</td>
<td class="c4t" colspan="4">4年</td>
<td class="c5t" colspan="4">5年</td>
</tr>
<tr>
<td class="c1t" colspan="2">前</td>
<td class="c1t" colspan="2">後</td>
<td class="c2t" colspan="2">前</td>
<td class="c2t" colspan="2">後</td>
<td class="c3t" colspan="2">前</td>
<td class="c3t" colspan="2">後</td>
<td class="c4t" colspan="2">前</td>
<td class="c4t" colspan="2">後</td>
<td class="c5t" colspan="2">前</td>
<td class="c5t" colspan="2">後</td>
</tr>
<tr>
<td class="c1t">1Q</td>
<td class="c1t">2Q</td>
<t

### ？個目の`table`から`tr`タグを抽出
- いくつ抽出された？
- 柱の部分は何行ある（いくつの`tr`からなる）？

In [8]:
trs = tables[2].find_all('tr')
len(trs)

142

In [9]:
trs[4]

<tr class="courseSubject" data-course-value="副専攻　グローバル系(必修)">
<td class="c1">一般</td>
<td>選択</td>
<td>
<div class="subject-item" id="subject_0004">
<a class="mcc-show" href="/Pages/PublicSyllabus?school_id=11&amp;department_id=12&amp;subject_id=0004&amp;year=2024&amp;lang=ja">Global Science</a>
<span class="mcc-hide">Global Science</span>
</div>
</td>
<td>0004</td>
<td>履修単位</td>
<td>2</td>
<td class="c1m" colspan="2"></td>
<td class="c1m" style="display:none;"></td>
<td class="c1m" colspan="2"></td>
<td class="c1m" style="display:none;"></td>
<td class="c2m" colspan="2">2</td>
<td class="c2m" style="display:none;"></td>
<td class="c2m" colspan="2">2</td>
<td class="c2m" style="display:none;"></td>
<td class="c3m" colspan="2"></td>
<td class="c3m" style="display:none;"></td>
<td class="c3m" colspan="2"></td>
<td class="c3m" style="display:none;"></td>
<td class="c4m" colspan="2"></td>
<td class="c4m" style="display:none;"></td>
<td class="c4m" colspan="2"></td>
<td class="c4m" style="dis

### 表の柱以外から各行(`tr`)ごとに`td`タグを抽出
- 各行の要素数（`td`の数）がすべて同じことを確認

In [10]:
elems = [tr.find_all('td') for tr in trs[4:]]
{len(x) for x in elems}

{28}

In [11]:
elems[0]

[<td class="c1">一般</td>,
 <td>選択</td>,
 <td>
 <div class="subject-item" id="subject_0004">
 <a class="mcc-show" href="/Pages/PublicSyllabus?school_id=11&amp;department_id=12&amp;subject_id=0004&amp;year=2024&amp;lang=ja">Global Science</a>
 <span class="mcc-hide">Global Science</span>
 </div>
 </td>,
 <td>0004</td>,
 <td>履修単位</td>,
 <td>2</td>,
 <td class="c1m" colspan="2"></td>,
 <td class="c1m" style="display:none;"></td>,
 <td class="c1m" colspan="2"></td>,
 <td class="c1m" style="display:none;"></td>,
 <td class="c2m" colspan="2">2</td>,
 <td class="c2m" style="display:none;"></td>,
 <td class="c2m" colspan="2">2</td>,
 <td class="c2m" style="display:none;"></td>,
 <td class="c3m" colspan="2"></td>,
 <td class="c3m" style="display:none;"></td>,
 <td class="c3m" colspan="2"></td>,
 <td class="c3m" style="display:none;"></td>,
 <td class="c4m" colspan="2"></td>,
 <td class="c4m" style="display:none;"></td>,
 <td class="c4m" colspan="2"></td>,
 <td class="c4m" style="display:none;"></

### 数行を調べて、必要な情報が何番目のカラム(`td`)に格納されているかを調査
- カラム番号(0～)とテキストを出力
- 調査結果を記入してください
  - 科目名：第？カラム
  - 学年：第？～？カラム
  - 学期：第？～？カラム
  - 科目区分：第？カラム
  - 必須/選択：第？カラム
  - 履修/学修：第？カラム
  - 単位数：第？カラム

In [12]:
line = elems[0] # 最初の科目（国語IA）
for i in range(len(line)) :
  print(i, line[i].get_text())

0 一般
1 選択
2 

Global Science
Global Science


3 0004
4 履修単位
5 2
6 
7 
8 
9 
10 2
11 
12 2
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
                                        二田 亜弥
                                    
27 



科目名：`get_text()`は複数行を抽出するので、
行`line`から科目名を抽出する関数を作ってみよう
- 何カラム目？
- 前回の演習（科目名の抽出）を参照

In [13]:
def getSubject(line) :
  return line[2].find('a').get_text()

In [14]:
# 確認テスト
print(getSubject(elems[1]))

体育実技Ⅰ


学年別週当授業時数
- {1|2|3|4|5}の{前|後}（10個）は第？カラム
- 10科目を選んで調べる

In [15]:
subj = ['国語IA', '国語IB', '国語IIA','国語IIB','国語III','現代社会B','体育II','英語演習IB','英語演習II','体育III'] # 5×2=10科目を選ぶ
for s in subj : # 科目を1個ずつ
  for line in elems : # 行ごとに調べ
    if s == getSubject(line) : # その科目がある行を見つけて
      for i in range(6,26) : # 「学年別週当時数」のカラムを1個ずつ
        d = line[i].get_text() # テキストを抽出して
        if d != '' : # 空欄でなければ
          print(i, s, d) # カラム番号、科目名、抽出したテキストを出力

### 例外に関する対応策が必要：半期科目でない科目
- 前期と後期をまとめて「集中講義」と記載：「日本文化論」（３箇所）
- 前期と後期に授業時数が記載：「卒業研究」など

それぞれ1科目を選んで調べる
- 調べ方は、前述の「10科目」と同様

In [16]:
subj = ['日本文化論','卒論研究'] # 2科目を選ぶ
for s in subj : # 科目を1個ずつ
  for line in elems : # 行ごとに調べ
    if s == getSubject(line) : # その科目がある行を見つけて
      for i in range(6,26) : # 「学年別週当時数」のカラムを1個ずつ
        d = line[i].get_text() # テキストを抽出して
        if d != '' : # 空欄でなければ
          print(i, s, d) # カラム番号、科目名、抽出したテキストを出力

### 学年と学期の抽出

行`line`から学年と学期を求めるアルゴリズム
- 学年別週当授業時数（？カラムから？カラムまで）に文字列があるカラムを探す
- `i`番目のカラムにあれば、
  - 学年の計算式：？？
  - 前期/後期/通年/集中講義の判定方法：
    - 前期：？？
    - 後期：？？
    - 通年：？？
    - 集中講義：？？


In [17]:
def getPeriod(line) :
  d = [(i-6)/4+1 for i in range(6,26) if line[i].get_text()!= ''] # 空欄でないセルの学年をリスト化
  grade = int(d[0]) # 学年を決定
  if len(d)==2 : # 通年
    ptype = '通年'
  elif len(d)==4 : # 集中講義
    ptype = '集中講義'
  elif d[0]==grade : # 前期
    ptype = '前期'
  else : # 後期
    ptype = '後期'
  return (grade, ptype)

In [18]:
# 確認テスト
for line in elems :
  print(getSubject(line), getPeriod(line))


Global Science (2, '通年')
体育実技Ⅰ (2, '通年')
グローバル研修 (2, '集中講義')
社会貢献 (2, '通年')
国語Ⅱ (2, '通年')
日本史 (2, '通年')
英語Ⅱ (2, '通年')
Oral Communication (2, '通年')
代数・幾何 (2, '通年')
解析学 (2, '通年')
化学 (2, '通年')
物理 (2, '通年')
電気電子基礎学 (2, '通年')
化学通論Ⅰ (2, '通年')
電気回路 (2, '通年')
コンピュータプログラミングⅠ (2, '通年')
論理回路 (2, '通年')
プログラミングⅠ (2, '後期')
機械・制御工学実験 (2, '通年')
機械設計製図基礎 (2, '通年')
加工工学 (2, '通年')
体育実技Ⅰ (3, '通年')
グローバル研修 (3, '集中講義')
社会貢献 (3, '通年')
国語Ⅲ (3, '通年')
世界史 (3, '通年')
英語Ⅲ (3, '通年')
Oral Communication (3, '通年')
実践英語 (3, '前期')
日本語Ⅲ (3, '通年')
代数・幾何 (3, '通年')
解析学 (3, '通年')
Global Presentation (3, '前期')
Global Writing (3, '後期')
化学通論Ⅱ (3, '通年')
電気電子回路基礎 (3, '通年')
コンピュータプログラミングⅡ (3, '通年')
機械設計製図基礎 (3, '通年')
工業力学 (3, '通年')
電気回路 (3, '通年')
基礎材料力学 (3, '通年')
電子回路 (3, '通年')
電磁気学Ⅰ (3, '後期')
プログラミングⅡ (3, '前期')
機械・制御工学実験 (3, '通年')
応用物理Ⅰ (3, '通年')
社会貢献 (4, '集中講義')
グローバル研修 (4, '集中講義')
キャリアデザイン (4, '集中講義')
経済概論 (4, '前期')
経営概論 (4, '前期')
経済概論 (4, '後期')
経営概論 (4, '後期')
現代の社会Ⅰ (4, '通年')
現代の社会Ⅱ (4, '通年')
歴史と文化Ⅰ (4, '通年')
人間と世界Ⅰ (4, '通年')


### すべての行から情報を抽出してリストにまとめる
- リストの要素：１科目の情報７個をリストにまとめる
  - 科目名
  - 学年
  - 前期/後期/通年など
  - 一般/専門
  - 必須/選択/必須選択など
  - 単位種別
  - 単位数

In [24]:
info = [[getSubject(line),getPeriod(line)[0],getPeriod(line)[1],line[0].get_text(),line[1].get_text(),line[4].get_text(),line[5].get_text()] for line in elems] # 7項目をリスト化
info

[['Global Science', 2, '通年', '一般', '選択', '履修単位', '2'],
 ['体育実技Ⅰ', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['グローバル研修', 2, '集中講義', '一般', '選択', '履修単位', '1'],
 ['社会貢献', 2, '通年', '一般', '選択', '履修単位', '1'],
 ['国語Ⅱ', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['日本史', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['英語Ⅱ', 2, '通年', '一般', '必修', '履修単位', '4'],
 ['Oral Communication', 2, '通年', '一般', '必修', '履修単位', '1'],
 ['代数・幾何', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['解析学', 2, '通年', '一般', '必修', '履修単位', '3'],
 ['化学', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['物理', 2, '通年', '一般', '必修', '履修単位', '2'],
 ['電気電子基礎学', 2, '通年', '専門', '選択', '履修単位', '2'],
 ['化学通論Ⅰ', 2, '通年', '専門', '選択', '履修単位', '2'],
 ['電気回路', 2, '通年', '専門', '必修', '履修単位', '1'],
 ['コンピュータプログラミングⅠ', 2, '通年', '専門', '選択', '履修単位', '2'],
 ['論理回路', 2, '通年', '専門', '必修', '履修単位', '1'],
 ['プログラミングⅠ', 2, '後期', '専門', '必修', '履修単位', '1'],
 ['機械・制御工学実験', 2, '通年', '専門', '必修', '履修単位', '3'],
 ['機械設計製図基礎', 2, '通年', '専門', '必修', '履修単位', '2'],
 ['加工工学', 2, '通年', '専門', '必修', '履修単位', '2'],
 ['体育

## csvファイルを出力
- pandasライブラリの`DataFrame`型を利用する

### データを表形式に変換する

In [25]:
df = pd.DataFrame(info)
df

,0,1,2,3,4,5,6
0,Global Science,2,通年,一般,選択,履修単位,2
1,体育実技Ⅰ,2,通年,一般,必修,履修単位,2
2,グローバル研修,2,集中講義,一般,選択,履修単位,1
3,社会貢献,2,通年,一般,選択,履修単位,1
4,国語Ⅱ,2,通年,一般,必修,履修単位,2
...,...,...,...,...,...,...,...
133,機械・制御工学英語,5,前期,専門,選択,学修単位II,1
134,機械・制御工学実験,5,通年,専門,必修,履修単位,4
135,卒業研究,5,通年,専門,必修,履修単位,9
136,Physical Mathmatics,5,後期,専門,選択,学修単位II,2


### カラム名を付ける

In [26]:
df.columns = ['科目名','学年','学期','一般/専門','必須/選択','履修/学習','単位数'] # カラム名7個
df

,科目名,学年,学期,一般/専門,必須/選択,履修/学習,単位数
0,Global Science,2,通年,一般,選択,履修単位,2
1,体育実技Ⅰ,2,通年,一般,必修,履修単位,2
2,グローバル研修,2,集中講義,一般,選択,履修単位,1
3,社会貢献,2,通年,一般,選択,履修単位,1
4,国語Ⅱ,2,通年,一般,必修,履修単位,2
...,...,...,...,...,...,...,...
133,機械・制御工学英語,5,前期,専門,選択,学修単位II,1
134,機械・制御工学実験,5,通年,専門,必修,履修単位,4
135,卒業研究,5,通年,専門,必修,履修単位,9
136,Physical Mathmatics,5,後期,専門,選択,学修単位II,2


### csvファイルとして出力
- pandasライブラリを用いて、`DataFrame`型データをcsvファイルに書き込む
- index（通し番号）は出力しない
- 文字化けを防ぐ

In [27]:
csvfile = 'results.csv' # ファイル名
df.to_csv(csvfile,index=False)